# Tutorial: Combinators (One-Shot `evaluate(...)`)

This notebook recreates examples from the official BQN tutorial page:
https://mlochbaum.github.io/BQN/tutorial/combinator.html

This is the one-shot variant: each line is an independent `evaluate("...")` call.


In [ ]:
from pathlib import Path
import sys


def _find_repo_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if (candidate / "pyproject.toml").exists() and (candidate / "src" / "bqn_jax").exists():
            return candidate
    raise RuntimeError(
        "Could not locate the bqn-jax repo root. Start Jupyter from this repo or set PYTHONPATH=src."
    )


_repo_root = _find_repo_root(Path.cwd())
_src = _repo_root / "src"
if str(_src) not in sys.path:
    sys.path.insert(0, str(_src))

import bqn_jax
from bqn_jax import evaluate

print(f"Using bqn_jax from: {bqn_jax.__file__}")


## Before You Start

1. Run the setup cell first (`from bqn_jax import evaluate`).
2. Then run cells from top to bottom.
3. Each `evaluate("...")` call parses and runs only that snippet.
4. Some advanced tutorial examples may raise errors because `bqn_jax` does not implement every BQN feature yet.


## What's a combinator?

Combinators build new functions from existing ones without introducing explicit variables.


In [ ]:
evaluate('|∘- 6')


In [ ]:
evaluate('7 |∘- 9')


In [ ]:
evaluate('14‿8 |∘- 19‿6')
evaluate('14‿8 +´∘|∘- 19‿6')


## Comparisons

Use these examples to see scalar and elementwise boolean comparisons.


In [ ]:
evaluate('3 < 4')
evaluate('4 > ∞')
evaluate('∞ < @')


In [ ]:
evaluate('\'e\' = "George Boole"')
evaluate('+´ \'e\' = "George Boole"')
evaluate('\'e\' +´∘= "George Boole"  # With a combinator')


In [ ]:
evaluate('"abcd" ×´∘= "abdd"')


In [ ]:
evaluate('"abcd" ≡ "abdd"')
evaluate('"abc"‿"de" ≡ "abc"‿"de"')


In [ ]:
evaluate('2‿3‿4‿2 ≠ 3‿3‿2‿2')
evaluate('2‿3‿4‿2 ≢ 3‿3‿2‿2')


## Length, rank, and depth

These primitives inspect structure: item count, dimensionality, and nesting depth.


In [ ]:
evaluate('≠ "testing"')
evaluate('≠ ⟨⟩')
evaluate('≠ ⟨ π, ∘, "element" ⋄ ⟨\'l\',1,5,\'t\'⟩ ⟩')
evaluate('≠ 4')


In [ ]:
evaluate('= 0.5')
evaluate('= ↕3')
evaluate("= 'a'")
evaluate('= "a"')


In [ ]:
evaluate('≡ "dream"                  # An ordinary dream')
evaluate('≡ "d"‿"r"‿"e"‿"a"‿"m"      # What if the letters were strings?')
evaluate('≡ ⟨ "d"‿"r"‿"e"‿"a"‿"m" ⟩  # We have to go deeper')


## Composition

Composition combines smaller transforms into larger reusable pipelines.


In [ ]:
evaluate('(≠"string") = ≠"sting"')


In [ ]:
evaluate('=´≠¨ ⟨"string","sting"⟩')


In [ ]:
evaluate('"string" =○≠ "sting"')


In [ ]:
evaluate('"string" ⋈○≠ "sting"')
evaluate('⋈○≠ "sting"')


In [ ]:
evaluate('2 ⋆⟜- 3')
evaluate('2 ⋆⊸- 3')


In [ ]:
evaluate('4 -⊸⌽ " before"  # Rotate to the right by four')
evaluate('4 ⌽⁼  " before"  # Okay this time Undo is better')


In [ ]:
evaluate('¬⊸× 0.5')


In [ ]:
evaluate('↕⊸÷ 8')
evaluate('¬⊸× ↕⊸÷ 8')


In [ ]:
evaluate('1⊸+ 5')
evaluate('+⟜1 5')


In [ ]:
evaluate('"const"˜ 5')
evaluate('@ "const"˜ 6')


In [ ]:
evaluate('+⊸1 5')


In [ ]:
evaluate('↕⊸÷ 8')
evaluate('(↕8) ÷ 7')


In [ ]:
evaluate('-⟜1 8')


In [ ]:
evaluate('↕⊸÷⟜(-⟜1) 8')


In [ ]:
evaluate('(↕÷-⟜1) 8')


## Base decoding continued

This revisits decoding and progressively compresses it into tacit combinator form.


In [ ]:
evaluate('@ + +´¨ (⌽2⋆↕8)⊸×¨ \'0\' -˜ "01000010"‿"01010001"‿"01001110"')


In [ ]:
evaluate('@⊸+ +´¨ (⌽2⋆↕8)⊸×¨ -⟜\'0\' "01000010"‿"01010001"‿"01001110"')
evaluate('(@⊸+)∘(+´¨)∘((⌽2⋆↕8)⊸×¨)∘(-⟜\'0\') "01000010"‿"01010001"‿"01001110"')


In [ ]:
evaluate('@⊸+∘(+´∘((⌽2⋆↕8)⊸×)¨)∘(-⟜\'0\') "01000010"‿"01010001"‿"01001110"')


In [ ]:
evaluate('@⊸+∘(+´∘×⟜(⌽2⋆↕8)¨)∘(-⟜\'0\') "01000010"‿"01010001"‿"01001110"')


In [ ]:
evaluate('(((((1×10)+2)×10)+3)×10)+4')
evaluate('((1×⟜10⊸+2)×⟜10⊸+3)×⟜10⊸+4   # Make the combining step a function')
evaluate('4+⟜(10⊸×)3+⟜(10⊸×)2+⟜(10⊸×)1 # Flip the combining function around')
evaluate("+⟜(10⊸×)´ 4‿3‿2‿1            # Now it's a BQN fold")
evaluate('+⟜(10⊸×)´ ⌽ 1‿2‿3‿4          # To fold in reverse, reverse then fold')


In [ ]:
evaluate('+´∘×⟜(⌽2⋆↕8) "01010001"-\'0\'')
evaluate('+⟜(2⊸×)´∘⌽ "01010001"-\'0\'')


In [ ]:
evaluate('@⊸+∘(+⟜(2⊸×)´∘⌽¨)∘(-⟜\'0\') "01000010"‿"01010001"‿"01001110"')


In [ ]:
evaluate('(@+ ·+⟜(2⊸×)´∘⌽¨ -⟜\'0\') "01000010"‿"01010001"‿"01001110"')


## Summary

If any result is unclear, rerun the section above step by step before moving on.
